## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-15-22-48-47 +0000,nyt,"In Munich, Lawmakers Concede Scars Remain Afte...",https://www.nytimes.com/2026/02/15/us/politics...
1,2026-02-15-22-34-09 +0000,nyt,Canada Gives U.S. Arms Makers the Cold Shoulde...,https://www.nytimes.com/2026/02/15/world/canad...
2,2026-02-15-22-30-18 +0000,nypost,"Flames tear through NYC apt., leaving 3 injure...",https://nypost.com/2026/02/15/us-news/flames-t...
3,2026-02-15-22-28-49 +0000,nypost,$525K flag that covered Abraham Lincoln’s cask...,https://nypost.com/2026/02/15/us-news/525k-fla...
4,2026-02-15-22-21-57 +0000,bbc,What we know about the abduction of Savannah G...,https://www.bbc.com/news/articles/cdex1nlrj07o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2516/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,11
74,new,8
57,will,7
64,epstein,7
93,europe,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...,34
140,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...,34
73,2026-02-15-15-32-07 +0000,bbc,'Trump will be gone in three years': Top Democ...,https://www.bbc.com/news/articles/cjrq2r9y278o...,34
16,2026-02-15-21-03-28 +0000,nypost,Multiple additional gloves found by The Post i...,https://nypost.com/2026/02/15/us-news/multiple...,33
80,2026-02-15-14-04-31 +0000,nyt,In First Public Comments Since Trump’s Racist ...,https://www.nytimes.com/2026/02/15/us/politics...,28


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,34,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...
140,34,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...
16,33,2026-02-15-21-03-28 +0000,nypost,Multiple additional gloves found by The Post i...,https://nypost.com/2026/02/15/us-news/multiple...
80,28,2026-02-15-14-04-31 +0000,nyt,In First Public Comments Since Trump’s Racist ...,https://www.nytimes.com/2026/02/15/us/politics...
147,27,2026-02-14-23-21-36 +0000,nypost,Doctors Without Borders halts operations in Ha...,https://nypost.com/2026/02/14/world-news/docto...
137,26,2026-02-15-02-04-02 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
92,26,2026-02-15-13-08-14 +0000,nyt,"Gisèle Pelicot, the Woman at the Center of Rap...",https://www.nytimes.com/2026/02/13/magazine/gi...
124,24,2026-02-15-05-50-08 +0000,nypost,DOJ sends Congress list of names who appear in...,https://nypost.com/2026/02/15/us-news/doj-send...
33,23,2026-02-15-19-58-47 +0000,nypost,Shocking homophobic display by high school stu...,https://nypost.com/2026/02/15/us-news/redwood-...
44,23,2026-02-15-18-48-38 +0000,nypost,Schumer says there are 3 sticking points in DH...,https://nypost.com/2026/02/15/us-news/chuck-sc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
